损失函数之回归损失
===

# 1.均方误差-Mean Square Error
也可以叫做二次损失(Quadratic Loss)，或者L2损失(L2 Loss)。MSE是目标变量与预测值之间的距离平方和
$$MSE=\frac{\sum_{i=1}^n(y_i-y_i^p)^2}{n}$$

![images](images/13_02_001.jpg)

In [1]:
import numpy as np

def mse(true, pred):
    return np.sum((true - pred) ** 2)

# 2.平均绝对误差-Mean Absolute Error
也可以叫做L1损失(L1 Loss).MAE是目标变量和预测变量之间差异绝对值之和。因此，它在一组预测中衡量误差的平均大小，而不考虑误差的方向。(如果我们也考虑方向，那将被称为平均偏差(Mean Bias Error, MBE)，它是残差或误差之和)
$$MAE=\frac{\sum_{i=1}^n\left| y_i-y_i^p \right|}{n}$$

![images](images/13_02_002.jpg)

简而言之， 使用平方误差更容易求解，但使用绝对误差对离群点更加鲁棒。每当我们训练机器学习模型时，我们的目标就是找到最小化损失函数的点。当然，当预测值正好等于真实值时，这两个损失函数都达到最小值。

In [3]:
import numpy as np

def mae(true, pred):
    return np.sum(np.abs(true - pred))

# 3.MAE与MSE的比较

由于MSE对误差$\epsilon$进行平方操作($y - y_predicted = \epsilon$)，如果$\epsilon$ > 1，误差的值会增加很多。如果我们的数据中有一个离群点，$\epsilon$的值将会很高，将会远远大于$\left| \epsilon \right|$。这将使得和以MAE为损失的模型相比，以MSE为损失的模型会赋予更高的权重给离群点。以MSE为损失的模型将被调整以最小化这个离群数据点，但是却是以牺牲其他正常数据点的预测效果为代价，这最终会降低模型的整体性能。

对所有的观测数据，如果我们只给一个预测结果来最小化MSE，那么该预测值应该是所有目标值的均值。但是如果我们试图最小化MAE，那么这个预测就是所有目标值的中位数。我们知道中位数对于离群点比平均值更鲁棒，这使得MAE比MSE更加鲁棒。

使用MAE损失(特别是对于神经网络)的一个大问题是它的梯度始终是相同的，这意味着即使对于小的损失值，其梯度也是大的。这对模型的学习可不好。为了解决这个问题，我们可以使用随着接近最小值而减小的动态学习率。MSE在这种情况下的表现很好，即使采用固定的学习率也会收敛。MSE损失的梯度在损失值较高时会比较大，随着损失接近0时而下降，从而使其在训练结束时更加精确。如果离群点是会影响业务、而且是应该被检测到的异常值，那么我们应该使用MSE。另一方面，如果我们认为离群点仅仅代表数据损坏，那么我们应该选择MAE作为损失。

但是对于这两种模型来说，MAE更加关注的是大部分不离群的情况，而忽略了少部分离群的情况，因为他会尝试去接近中值；但是MSE则会关注离群的情况，因为它被离群点弄糊涂了。所以这两种情况在许多业务中都是不可取的。

| L2损失函数 | L1损失函数 |
| --------- | -------- |
| 不是非常的鲁棒 | 鲁棒 |
| 稳定解 | 不稳定解 |
| 总是一个解 | 可能多个解 |

# 4.平滑的平均绝对误差-Huber Loss
Huber Loss对数据离群点的敏感度低于平方误差损失。它在0处也可导。基本上它是绝对误差，当误差很小时，误差是二次形式的。误差何时需要变成二次形式取决于一个超参数$\delta$，该超参数可以进行微调。当$\delta \sim 0$时， Huber Loss接近MAE，当$\delta \sim \infty$（很大的数）时，Huber Loss接近MSE
$$
L_{\delta}(y, f(x))=\begin{cases}
\frac{1}{2}(y-f(x))^2 &\left| y-f(x) \right| \leq \delta \\
\delta \left| y-f(x) \right|-\frac{1}{2}\delta^2 &otherwise
\end{cases}
$$

![images](images/13_02_003.jpg)

使用MAE训练神经网络的一个大问题是经常会遇到很大的梯度，使用梯度下降时可能导致训练结束时错过最小值。对于MSE，梯度会随着损失接近最小值而降低，从而使其更加精确。在这种情况下，Huber Loss可能会非常有用，因为它会使最小值附近弯曲，从而降低梯度。另外它比MSE对异常值更鲁棒。因此，它结合了MSE和MAE的优良特性。但是，Huber Loss的问题是我们可能需要迭代地训练超参数$\delta$。

# 5.Log-Cosh Loss
Log-cosh是用于回归任务的另一种损失函数，它比L2更加平滑。Log-cosh是预测误差的双曲余弦的对数
$$L(y,y^p)=\sum_{i=1}^nlog[cosh(y_i^p-y_i)]$$

![images](images/13_02_004.jpg)

$log(cosh(x))$对于小的$x$来说，其大约等于$\frac{x^2}{2}$，而对于大的$x$来说，其大约等于$abs(x) - log(2)$。这意味着'logcosh'的作用大部分与均方误差一样，但不会受到偶尔出现的极端不正确预测的强烈影响。它具有Huber Loss的所有优点，和Huber Loss不同之处在于，其处处二次可导。为什么我们需要二阶导数？许多机器学习模型的实现（如XGBoost）使用牛顿方法来寻找最优解，这就是为什么需要二阶导数（Hessian）的原因。对于像XGBoost这样的机器学习框架，二阶可导函数更有利。

# 6.smooth L1 Loss
smooth L1说的是光滑之后的L1
$$
smooth_{L1}(y_i, \hat{y_i})=\begin{cases}
\frac{1}{2}(y_i-\hat{y_i})^2 & \left| y_i-\hat{y_i} \right| < 1 \\
\left| y_i-\hat{y_i} \right| - 0.5 & otherwise
\end{cases}
$$

![images](images/13_02_005.png)

误差在$(-1,1)$上是平方损失，其他情况是L1损失

In [4]:
import torch
loss_fn = torch.nn.SmoothL1Loss(reduce=False, size_average=False)
input = torch.autograd.Variable(torch.randn(3,4))
target = torch.autograd.Variable(torch.randn(3,4))
loss = loss_fn(input, target)
print(input); print(target); print(loss)
print(input.size(), target.size(), loss.size())

tensor([[ 1.1680, -0.7531, -0.9558, -0.6298],
        [ 0.8645, -0.6154, -0.0812, -1.1436],
        [-1.0007,  0.1825, -0.0731,  0.8301]])
tensor([[ 0.3198,  0.3127,  2.4066, -0.7760],
        [ 0.7471, -1.4110,  0.7873,  1.5462],
        [-0.4629,  0.2484, -1.3869,  0.6101]])
tensor([[3.5973e-01, 5.6576e-01, 2.8624e+00, 1.0698e-02],
        [6.8895e-03, 3.1647e-01, 3.7713e-01, 2.1898e+00],
        [1.4458e-01, 2.1701e-03, 8.1389e-01, 2.4200e-02]])
torch.Size([3, 4]) torch.Size([3, 4]) torch.Size([3, 4])
